<a href="https://colab.research.google.com/github/kaipak/from-eyes-to-ears/blob/master/mask_training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install numpy scikit-image Shapely awscli

In [0]:
import pandas as pd
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import pickle

In [230]:
!rm -rf train validation test *.zip
!mkdir tmp
!mkdir train
!mkdir train/images
!mkdir validation
!mkdir validation/images
!mkdir test
!mkdir test/images
!mkdir masks/

mkdir: cannot create directory ‘tmp’: File exists
mkdir: cannot create directory ‘masks/’: File exists


In [0]:
phase = 'train'
filter_categories = ["Saxophone", "Tank", "Goose", "Guitar"]

## Run the cell that match the current phase

In [219]:
!rm *.csv
!wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
!wget https://storage.googleapis.com/openimages/v5/train-annotations-human-imagelabels-boxable.csv
!rm -rf masks/
!mkdir masks/
%cd masks/
!wget https://storage.googleapis.com/openimages/v5/train-masks/train-masks-0.zip
!unzip train-masks-0.zip > /dev/null
!wget https://storage.googleapis.com/openimages/v5/train-masks/train-masks-1.zip
!unzip train-masks-1.zip > /dev/null
%cd ..

--2020-05-06 00:29:14--  https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c0a::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12011 (12K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv’

class-descriptions- 100%[===================>]  11.73K  --.-KB/s    in 0s      

2020-05-06 00:29:14 (92.3 MB/s) - ‘class-descriptions-boxable.csv’ saved [12011/12011]

--2020-05-06 00:29:15--  https://storage.googleapis.com/openimages/v5/train-annotations-human-imagelabels-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 376764810 (359M) [text/csv]
Saving to: ‘trai

In [195]:
!rm *.csv
!wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
!wget https://storage.googleapis.com/openimages/v5/validation-annotations-human-imagelabels-boxable.csv
!rm -rf masks/
!mkdir masks/
%cd masks/
!wget https://storage.googleapis.com/openimages/v5/validation-masks/validation-masks-0.zip
!unzip validation-masks-0.zip > /dev/null
!wget https://storage.googleapis.com/openimages/v5/validation-masks/validation-masks-1.zip
!unzip validation-masks-1.zip > /dev/null
%cd ..

--2020-05-05 23:35:24--  https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12011 (12K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv’

class-descriptions- 100%[===================>]  11.73K  --.-KB/s    in 0s      

2020-05-05 23:35:24 (68.5 MB/s) - ‘class-descriptions-boxable.csv’ saved [12011/12011]

--2020-05-05 23:35:25--  https://storage.googleapis.com/openimages/v5/validation-annotations-human-imagelabels-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10649275 (10M) [text/csv]
Saving to: 

In [204]:
!rm *.csv
!wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
!wget https://storage.googleapis.com/openimages/v5/test-annotations-human-imagelabels-boxable.csv
!rm -rf masks/
!mkdir masks/
%cd masks/
!wget https://storage.googleapis.com/openimages/v5/test-masks/test-masks-0.zip
!unzip test-masks-0.zip > /dev/null
!wget https://storage.googleapis.com/openimages/v5/test-masks/test-masks-1.zip
!unzip test-masks-1.zip > /dev/null
%cd ..

--2020-05-05 23:36:57--  https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12011 (12K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv’

class-descriptions- 100%[===================>]  11.73K  --.-KB/s    in 0s      

2020-05-05 23:36:57 (88.8 MB/s) - ‘class-descriptions-boxable.csv’ saved [12011/12011]

--2020-05-05 23:36:58--  https://storage.googleapis.com/openimages/v5/test-annotations-human-imagelabels-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32055857 (31M) [text/csv]
Saving to: ‘test-an

In [0]:
classes = pd.read_csv('class-descriptions-boxable.csv', header=None, names=["LabelName", "class"])
image_classes = pd.read_csv(f'{phase}-annotations-human-imagelabels-boxable.csv')

In [0]:
# Find where we have data
training_imageids = classes.join(image_classes.set_index('LabelName'), on='LabelName').filter(["ImageID", "LabelName","class"])
training_imageids = training_imageids[training_imageids["ImageID"].str.startswith('0') | training_imageids["ImageID"].str.startswith('1')]
training_imageids['cutLabelName'] = training_imageids["LabelName"].str.replace('/', '')
masks = pd.DataFrame([f for f in [file.split('_') + [file] for file in os.listdir('masks')] if len(f) == 4], columns=['ImageID', 'cutLabelName', 'extension', 'mask_filename'])
training_imageids = training_imageids.merge(masks,  how='inner', left_on=['ImageID', 'cutLabelName'], right_on = ['ImageID', 'cutLabelName']).filter(["ImageID", "LabelName", "class", 'mask_filename'])

## Get a summary to pick classes for training

In [222]:
i = pd.get_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
print(training_imageids.groupby("class").count()[['ImageID']])
pd.set_option('display.max_rows', i)

                         ImageID
class                           
Adhesive tape                 62
Aircraft                     217
Airplane                    2704
Alarm clock                   30
Alpaca                       143
Ambulance                    120
Apple                        521
Armadillo                      1
Artichoke                     40
Axe                           31
Backpack                     421
Balance beam                  58
Ball                         497
Balloon                     1114
Banana                       267
Band-aid                       2
Barge                        119
Barrel                       345
Baseball bat                 260
Baseball glove               331
Bat (Animal)                 121
Beaker                        36
Bear                          62
Beer                        1060
Bell pepper                  154
Belt                          68
Bicycle wheel               6542
Billiard table               227
Bird      

In [223]:
filtered = training_imageids[training_imageids["class"].isin(filter_categories)].filter(["ImageID", "mask_filename", "class"])
filtered["info"] = list(zip(filtered['mask_filename'], filtered['class']))
masks = filtered.groupby("ImageID")["info"].apply(list).values.tolist()
print(len(masks))

3370


In [227]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
categories = []
metadata = []
max_size = 10
cat_counts = {}
n = 1
for m in masks:
    annotations = []
    catset = set()
    for f, c in m:
        catset.add(c)
    
    cont = False
    for c in catset:
        if c in cat_counts:
            if cat_counts[c] > max_size:
                cont = True
                break
            else:
                cat_counts[c] += 1
        else:
            cat_counts[c] = 1
    
    if cont:
        #print(f"we have collected enough for categores {catset}")
        continue

    for file, category in m:
        if category not in categories:
            categories.append(category)

        category_id = categories.index(category)
        fullname = f"masks/{file}"
        image_id = file.split('_')[0]
        mask = (cv2.imread(fullname)[...,0]/255.0).astype(np.uint8)
        contours = measure.find_contours(mask, 0.5, positive_orientation='low')
        segmentations = []
        polygons = []
        for contour in contours:
            # Flip from (row, col) representation to (x, y)
            # and subtract the padding pixel
            for i in range(len(contour)):
                row, col = contour[i]
                contour[i] = (col - 1, row - 1)
            try:
                # Make a polygon and simplify it
                poly = Polygon(contour)
                poly = poly.simplify(1.0, preserve_topology=False)
                if poly.type == 'MultiPolygon':
                    for p in poly:
                        polygons.append(p)
                        segmentation = np.array(p.exterior.coords).ravel().tolist()
                        segmentations.append(segmentation)
                else:
                    polygons.append(poly)
                    segmentation = np.array(poly.exterior.coords).ravel().tolist()
                    segmentations.append(segmentation)
            except:
                print("Error processing image mask, skipping")
                continue
        
        # Combine the polygons to calculate the bounding box and area
        multi_poly = MultiPolygon(polygons)
        x, y, max_x, max_y = multi_poly.bounds
        width = max_x - x
        height = max_y - y
        bbox = (x, y, width, height)
        area = multi_poly.area

        annotation = {
            'bbox': multi_poly.bounds,
            'bbox_mode': 0,
            'category_id': category_id,
            'iscrowd': 0,
            'segmentation': segmentations
        }
        annotations.append(annotation)

    if len(annotations) > 0:
        h, w = mask.shape
        md = {
            'annotations': annotations,
            'file_name': f'train/images/{image_id}.jpg',
            'height': h,
            'width': w,
            'image_id': image_id
        }
        metadata.append(md)

        image_file = f'{image_id}.jpg'
        print(f"{n} - Fetching image: {image_file} - Categories: {catset} - Current Counters: {cat_counts}")
        s3.download_file('open-images-dataset', f'{phase}/{image_file}', f'tmp/{image_file}')
        img = cv2.imread(f'tmp/{image_file}')
        os.unlink(f'tmp/{image_file}')
        img = cv2.resize(img, (w, h))
        cv2.imwrite(f'{phase}/images/{image_file}', img)
        n += 1


print(f"Data ready for phase: {phase}")
print("Encoding Metadata")
pickle.dump({"categories": categories, "metadata": metadata}, open(f"{phase}/metadata.pkl", "wb"))
print("Done")

1 - Fetching image: 0000201cd362f303.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 1}
2 - Fetching image: 0000aa810854dc2e.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 2}
3 - Fetching image: 0000b3e5921ab7ff.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 3}
4 - Fetching image: 0000c33c6f4b8518.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 4}
5 - Fetching image: 0001ff6b8fc43d43.jpg - Categories: {'Goose'} - Current Counters: {'Guitar': 4, 'Goose': 1}
6 - Fetching image: 00027ae34cb09507.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 5, 'Goose': 1}
7 - Fetching image: 0002a3c01c926a49.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 6, 'Goose': 1}
8 - Fetching image: 00034cd6adabef6a.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 7, 'Goose': 1}
9 - Fetching image: 00036d5834d932bf.jpg - Categories: {'Guitar'} - Current Counters: {'Guitar': 8, 'Goose': 1}
10 - Fetching image: 0004c03cb6520680.jpg

In [251]:
from pprint import pprint 
t = metadata[1]["annotations"][0]
print(t["bbox"], min(t["segmentation"][0]))

num_p = int(len(t["segmentation"][0])/2)
points = np.array(t["segmentation"][0]).reshape((num_p, 2))
print(points[:,0].min(), points[:,1].min(), points[:,0].max(), points[:,1].max())
pprint(points)

(937.5, 492.5, 1237.5, 657.5) 492.5
937.5 492.5 1237.5 657.5
array([[1022. ,  657.5],
       [1025.5,  654. ],
       [1025.5,  649. ],
       [1031. ,  643.5],
       [1036. ,  643.5],
       [1039. ,  640.5],
       [1053. ,  640.5],
       [1055. ,  638.5],
       [1061. ,  638.5],
       [1065. ,  643.5],
       [1073. ,  643.5],
       [1074. ,  645.5],
       [1077. ,  645.5],
       [1079. ,  643.5],
       [1084. ,  643.5],
       [1088. ,  638.5],
       [1094. ,  638.5],
       [1096. ,  643.5],
       [1099. ,  643.5],
       [1103. ,  647.5],
       [1168. ,  647.5],
       [1170. ,  645.5],
       [1184. ,  645.5],
       [1193. ,  638.5],
       [1199. ,  638.5],
       [1201. ,  640.5],
       [1202.5,  639. ],
       [1202.5,  635. ],
       [1199.5,  632. ],
       [1204.5,  628. ],
       [1204.5,  625. ],
       [1199.5,  623. ],
       [1199.5,  611. ],
       [1203. ,  607.5],
       [1208. ,  607.5],
       [1209.5,  606. ],
       [1209.5,  603. ],
       [1214.5

In [228]:
!zip -r train.zip train 
!ls -lh

updating: train/ (stored 0%)
updating: train/metadata.pkl (deflated 77%)
updating: train/images/ (stored 0%)
updating: train/images/077d8f96cc7cb1fa.jpg (deflated 1%)
updating: train/images/0670b1c550b5280b.jpg (deflated 0%)
updating: train/images/07e1fad8dff29191.jpg (deflated 0%)
updating: train/images/0000c33c6f4b8518.jpg (deflated 0%)
updating: train/images/003711cdbf03cdd7.jpg (deflated 0%)
updating: train/images/0138ee3e8bf160a0.jpg (deflated 0%)
updating: train/images/0279e931857ab928.jpg (deflated 0%)
updating: train/images/0321217112660a68.jpg (deflated 0%)
updating: train/images/08b6aeea096bcf53.jpg (deflated 1%)
updating: train/images/00058c84b5ef9dba.jpg (deflated 1%)
updating: train/images/008648e2459d0c60.jpg (deflated 1%)
updating: train/images/0c62bda56fee0b1e.jpg (deflated 0%)
updating: train/images/080530010355fc1a.jpg (deflated 0%)
updating: train/images/00424033a2e368a8.jpg (deflated 0%)
updating: train/images/04ba39f0b6431681.jpg (deflated 0%)
updating: train/image

In [229]:
from google.colab import drive
drive.mount('/content/drive')
!cp *.zip "drive/My Drive"
drive.flush_and_unmount()

Mounted at /content/drive
